In [2]:
from api_key import get_api_key_jef, get_api_key_sha, get_api_key_sup
import google.generativeai as genai
from vpn_control import windscribe
import time
import re
import pandas as pd
import numpy as np



In [17]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [12]:
passw = ""
LLM_MODEL = "gemini-1.5-flash" #gemini-1.5-pro-latest

In [13]:
def setup_prompt_env(passw, extra_instruct):


  genai.configure(api_key=get_api_key_sup(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 2500,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = f"""You must modify the log template such that it coveys the same infromation but is structred differently/uses different words. You can add or remove <*> wildcards which will contain variables later. Only use sensible replacements in the context. Additonally {extra_instruct}. You must provide a level 1, 2, 3 and 4 change of the original log, where 0 is no change and 4 is where every token/word and order can be changed. Pad each log with a begin and end token <B> <E> and give the change level [x] at the start. Find am example below of different change levels [x]
  
  INPUT: Node <*> experienced data loss at <*> due to network error

  OUTPUT:
  [0]: <B>Node <*> experienced data loss at <*> due to network error<E>
  [1]: <B>Node <*> lost stored data at <*> due to network error<E>
  [2]: <B>Node <*> experienced failure at <*> causing data loss<E>
  [3]: <B>Due to network error <*> node lost all stored data at time <*><E>
  [4]: <B>Severe node failure on <*> at <*> due to network disconnection causing data loss<E>
  
 Dont give extra text."""

  model = genai.GenerativeModel(model_name=LLM_MODEL,
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [14]:
def augment_template_LLM(message, convo, extra_instruct):
    message =  message 
    convo.send_message(message)
    suggestion = convo.last.text
    #print(suggestion)
    return suggestion

In [15]:
def setup_LLM_aug(passw, extra_instruct, message):
    convo_create = setup_prompt_env(passw, extra_instruct)
    res = augment_template_LLM(message, convo_create, extra_instruct)
    return res

In [20]:
setup_LLM_aug(passw, ".", "<B>PacketResponder <*> for block blk_<*> terminating<E>")

'[0]: <B>PacketResponder <*> for block blk_<*> terminating<E>\n[1]: <B>PacketResponder <*> terminating for block blk_<*><E>\n[2]: <B>PacketResponder <*> ending processing for block blk_<*><E>\n[3]: <B>Block blk_<*> processing terminated by PacketResponder <*><E>\n[4]: <B>PacketResponder <*> has stopped processing block blk_<*> due to an unknown event<E> \n'

In [3]:
tmep = pd.read_csv("data\HDFS_2k.log_templates_aug.csv")
df = tmep

In [ ]:

def augment_logs(df):
    for index, row in df.iterrows():
        time.sleep(10)
        augmented_logs = setup_LLM_aug(passw, ".", row['EventTemplate']).split('\n')
        for log in augmented_logs:
            # Check if the log contains the expected delimiter
            if ': ' in log:
                col_index, augmented_log = log.split(': ', 1)
                col_index = col_index.strip('[]')
                df.at[index, col_index] = augmented_log
    return df


df = augment_logs(df)

# Display the updated DataFrame
print(df)

In [ ]:
df

,EventId,EventTemplate,0,1,2,3,4
0,E1,<*>:<*> Served block blk_<*> to /<*>,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B><*>:<*> Delivered block blk_<*> to /<*> <E>,<B>Block blk_<*> served by <*> to /<*> <E>,<B>Block blk_<*> was delivered to /<*> by <*> <E>,<B>Successful block delivery of blk_<*> to /<*...
1,E2,<*>:<*> Starting thread to transfer block blk_...,<B><*>:<*> Starting thread to transfer block b...,<B>Initiating transfer of block blk_<*> from <...,<B>Transfer thread started for block blk_<*> f...,<B>A thread has been started to transfer block...,<B>Block blk_<*> transfer initiated from <*>:<...
2,E3,<*>:<*>:Got exception while serving blk_<*> to...,<B><*>:<*>:Got exception while serving blk_<*>...,<B><*>:<*>:Exception occurred while serving bl...,<B>Exception encountered on <*> while serving ...,<B>Block <*> could not be served to /<*> on <*...,<B>Error: Block <*> could not be delivered to ...
3,E4,BLOCK* ask <*>:<*> to delete blk_<*>,<B>BLOCK* ask <*>:<*> to delete blk_<*><E>,<B>BLOCK* requests <*>:<*> to delete block blk...,<B>BLOCK* requests deletion of block blk_<*> f...,<B>Deletion request for block blk_<*> sent to ...,<B>BLOCK* initiated a block deletion request f...
4,E5,BLOCK* ask <*>:<*> to replicate blk_<*> to dat...,<B>BLOCK* ask <*>:<*> to replicate blk_<*> to ...,<B>BLOCK* request to replicate block blk_<*> f...,<B>Replication request for block blk_<*> from ...,<B>BLOCK* received a request to replicate bloc...,<B>Data replication task initiated for block b...
5,E6,BLOCK* NameSystem.addStoredBlock: blockMap upd...,<B>BLOCK* NameSystem.addStoredBlock: blockMap ...,<B>BLOCK* NameSystem.addStoredBlock: blockMap ...,<B>BLOCK* NameSystem.addStoredBlock: blockMap ...,<B>NameSystem added block <*>:<*> to blk_<*> w...,<B>Block <*>:<*> with size <*> was added to bl...
6,E7,BLOCK* NameSystem.allocateBlock: /<*>/part-<*>...,<B>BLOCK* NameSystem.allocateBlock: /<*>/part-...,<B>NameSystem allocated block: /<*>/part-<*>. ...,<B>Block allocation: /<*>/part-<*>. blk_<*> by...,<B>NameSystem allocated block /<*>/part-<*> wi...,<B>Block allocation successful for /<*>/part-<...
7,E8,BLOCK* NameSystem.delete: blk_<*> is added to ...,<B>BLOCK* NameSystem.delete: blk_<*> is added ...,<B>Block <*> was added to the invalid set of <...,<B>NameSystem.delete added block <*> to the in...,<B>The NameSystem.delete operation added block...,<B>Block <*> was marked as invalid by NameSyst...
8,E9,Deleting block blk_<*> file /<*>/blk_<*>,<B>Deleting block blk_<*> file /<*>/blk_<*> <E>,<B>Deleting block file blk_<*> located at /<*>...,<B>Block file blk_<*> at /<*>/blk_<*> is being...,<B>File blk_<*> located at /<*>/blk_<*> is bei...,<B>Removal of block file blk_<*> from the bloc...
9,E10,PacketResponder <*> for block blk_<*> terminating,<B>PacketResponder <*> for block blk_<*> termi...,<B>PacketResponder <*> terminating for block b...,<B>PacketResponder <*> is terminating for bloc...,<B>Block blk_<*> is being terminated by Packet...,<B>PacketResponder <*> has stopped processing ...


In [ ]:
df.to_csv('data/augmented_templates_flash.csv', index=False)

In [ ]:
windscribe("disconnect")